# Imputación de datos faltantes
Vamos a completar datos faltantes en ambos Datasets.
1. Dataset: dataset_train.csv
   - Contiene 1600 registros que vamos a completar parcialmente
   - Dividiendo los datos de 'Defense Against the Dark Arts' entre -100 se obtienen los datos de 'Astronomy', debido a su correlación perfecta.
   - Esto nos permitirá recuperar la mayoría de los datos faltantes en 'Defense Against the Dark Arts'.
   - El resto de datos faltantes no intentaremos recuperarlos y esos registros se eliminaran antes de entrar en la fase de entrenamiento.
3. Dataset: dataset_test.csv
   - Contiene 400 registros, vamos a completar todos los datos faltantes de las features elegidas.
   - La columna de 'Defense Against the Dark Arts' se completa aprovechando su correlación perfecta con 'Astronomy'.
   - El resto de datos faltantes se estimarán usando el algoritmo k-nearest neighbors (KNN), k vecinos más cercanos.

In [2]:
import pandas as pd

# Establecemos la ruta y leemos el archivo convirtiéndolo en un DataFrame
file_train = '../datasets/dataset_train.csv'
df_train = pd.read_csv(file_train, index_col=0)

# Análogamente para los datos de test
file_test = '../datasets/dataset_test.csv'
df_test = pd.read_csv(file_test, index_col=0)

## Imputación de datos faltantes usando 'Astronomy'
Usamos la correlación perfecta r=-1 entre 'Astronomy' y 'Defense Against the Dark Arts' para realizar la imputación de los datos faltantes que se puedan imputar de 'Defense Against the Dark Arts' aprovechando esta característica.

Lo único que hay que hacer es tomar el dato de 'Astronomy' y multiplicar por -0.01 para lograr conseguir el dato faltante de  'Defense Against the Dark Arts'.

In [3]:
# Imputar 'Defense Against the Dark Arts' usando 'Astronomy'

def data_imputation_perfect_correlation(df):
    # Calcular las filas totales
    total = len(df)
    
    # Calcular las filas con datos no nulos antes de la imputación
    n1 = df['Defense Against the Dark Arts'].count()
    
    # Crear máscara para identificar:
    # 1. Valores nulos en Defense Against the Dark Arts
    # 2. Valores no nulos en Astronomy
    mask = (df['Defense Against the Dark Arts'].isna() & 
            df['Astronomy'].notna())
    
    # Imputar los valores usando la relación perfecta (Astronomy * -0.01)
    df.loc[mask, 'Defense Against the Dark Arts'] = df.loc[mask, 'Astronomy'] * -0.01
    
    # Calcular las filas con datos no nulos después de la imputación
    n2 = df['Defense Against the Dark Arts'].count()
    
    # Verificar el resultado
    print("Valores no nulos en Defense Against the Dark Arts:")
    print(f"Antes de imputación: {n1}/{total}")
    print(f"Después de imputación: {n2}/{total}")
    
    print(f"\nHemos recuperado {n2-n1} filas.")

# Aplicación a los dos DataFrame
print(f"{'='*13} Para el DataFrame train {'='*13}")
data_imputation_perfect_correlation(df_train)
print()
print("="*13, "Para el DataFrame test", "="*13)
data_imputation_perfect_correlation(df_test)

============= Para el DataFrame train =============
Valores no nulos en Defense Against the Dark Arts:
Antes de imputación: 1569/1600
Después de imputación: 1600/1600

Hemos recuperado 31 filas.

============= Para el DataFrame test =============
Valores no nulos en Defense Against the Dark Arts:
Antes de imputación: 392/400
Después de imputación: 399/400

Hemos recuperado 7 filas.


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1600 entries, 0 to 1599
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Hogwarts House                 1600 non-null   object 
 1   First Name                     1600 non-null   object 
 2   Last Name                      1600 non-null   object 
 3   Birthday                       1600 non-null   object 
 4   Best Hand                      1600 non-null   object 
 5   Arithmancy                     1566 non-null   float64
 6   Astronomy                      1568 non-null   float64
 7   Herbology                      1567 non-null   float64
 8   Defense Against the Dark Arts  1600 non-null   float64
 9   Divination                     1561 non-null   float64
 10  Muggle Studies                 1565 non-null   float64
 11  Ancient Runes                  1565 non-null   float64
 12  History of Magic               1557 non-null   float6

In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 0 to 399
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Hogwarts House                 0 non-null      float64
 1   First Name                     400 non-null    object 
 2   Last Name                      400 non-null    object 
 3   Birthday                       400 non-null    object 
 4   Best Hand                      400 non-null    object 
 5   Arithmancy                     387 non-null    float64
 6   Astronomy                      387 non-null    float64
 7   Herbology                      389 non-null    float64
 8   Defense Against the Dark Arts  399 non-null    float64
 9   Divination                     394 non-null    float64
 10  Muggle Studies                 390 non-null    float64
 11  Ancient Runes                  392 non-null    float64
 12  History of Magic               389 non-null    float64


## Eliminación de 'Astronomy'

In [6]:
# Eliminar la columna 'Astronomy' después de usarla para la imputación
df_train = df_train.drop('Astronomy', axis=1)
df_test = df_test.drop('Astronomy', axis=1)

## Imputación de los datos de 'test' con el algoritmo de los k-vecinos más próximos (KNN)
Imputación de datos faltantes con KNN usando solo filas completas

### 1. Preparación de los datos